In [0]:
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_train.csv
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_test.csv
from IPython import display
display.clear_output()
from fastai.tabular import *

In [0]:
# import pandas as pd, numpy as np, matplotlib.pyplot as plt
df = pd.read_csv('doctor_train.csv')
test_df = pd.read_csv('doctor_test.csv')

cols = ['age','Money','day','Time','Doctor_visits','last_visit',
        'cured_in','Profession','Status','edu','Irregular','residence',
        'prev_diagnosed','communication','Month','side_effects']
dep_var = ['Y']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_y = le.fit(df['Y'])
df['Y'] = le_y.transform(df['Y'])

In [0]:
df_D = pd.get_dummies(df[cols])
test_df_D = pd.get_dummies(test_df[cols])
df_D=df_D.fillna(0)
# df_D[df_D < 0] = 0
test_df_D=test_df_D.fillna(0)
# test_df_D[test_df_D < 0] = 0
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2
# select_feature = SelectKBest(chi2, k=1).fit(df_D, df['Y'])
# df_D=select_feature.transform(df_D)
# test_df_D=select_feature.transform(test_df_D)
# df_D = pd.DataFrame(df_D)
# test_df_D = pd.DataFrame(test_df_D)

In [4]:
import featuretools as ft
es = ft.EntitySet(id = 'doc')

es.entity_from_dataframe(entity_id = 'docs', dataframe = df_D, index = 'ID')
tr_fdf, tr_fc = ft.dfs( entityset=es, target_entity="docs", 
                          #features_only=True,
                          agg_primitives=["num_unique", "mean", "sum", "max", "min", "std", "n_most_common"],
                          trans_primitives=["diff", "absolute", "percentile"],
                          max_depth=3,
                          max_features=1000,
                          # chunk_size=4000,
                          verbose=True,)

es.entity_from_dataframe(entity_id = 'docs', dataframe = test_df_D, index = 'ID')
te_fdf, te_fc = ft.dfs( entityset=es, target_entity="docs", 
                          #features_only=True,
                          agg_primitives=["num_unique", "mean", "sum", "max", "min", "std", "n_most_common"],
                          trans_primitives=["diff", "absolute", "percentile"],
                          max_depth=3,
                          max_features=1000,
                          # chunk_size=4000,
                          verbose=True,)

2020-05-19 19:41:35,190 featuretools.entityset - WARNING    index ID not found in dataframe, creating new integer column
Built 204 features
Elapsed: 00:15 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks
2020-05-19 19:41:51,092 featuretools.entityset - WARNING    index ID not found in dataframe, creating new integer column
Built 204 features
Elapsed: 00:03 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 10/10 chunks


In [15]:
from imblearn.combine import SMOTETomek
smt = SMOTETomek(ratio='auto')
df_D_over, df_Y_over = smt.fit_resample(tr_fdf, df['Y'])
df_D_over = pd.DataFrame(df_D_over)
te_fdf.columns = df_D_over.columns

num_cal = df_D_over.columns.values
cat_col = []
df_D_over['Y'] = df_Y_over
dep_var = ['Y']

procs = [FillMissing, Categorify, Normalize]
fDF = df_D_over
fTDF = te_fdf
data = (TabularList.from_df(fDF, procs=procs, cont_names=num_cal, cat_names=cat_col)
          .split_by_idx(valid_idx=range(int(len(fDF)*0.8),len(fDF)))
          .label_from_df(cols=dep_var)
          .add_test(TabularList.from_df(fTDF, cont_names=num_cal, cat_names=cat_col, procs=procs))
          .databunch())
learn = tabular_learner(data, layers=[100, 90, 80, 70, 60, 50, 40, 30, 20, 10, 5], metrics=accuracy)
learn.fit_one_cycle(10)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


epoch,train_loss,valid_loss,accuracy,time
0,0.399022,0.368962,0.847645,00:15
1,0.302865,0.372954,0.814143,00:15
2,0.275601,0.136477,0.954335,00:15
3,0.237809,0.201235,0.930523,00:15
4,0.231066,0.223705,0.914545,00:15
5,0.203599,0.154605,0.936398,00:15
6,0.203381,0.134127,0.962272,00:15
7,0.191251,0.122706,0.954953,00:15
8,0.172168,0.138343,0.947428,00:15
9,0.166989,0.110000,0.957736,00:15


In [16]:
preds, _ = learn.get_preds(ds_type=DatasetType.Test)
pred_prob, pred_class = preds.max(1)

le_y = le.inverse_transform(pred_class)

output = pd.DataFrame({'id': test_df.ID,
                       'Y': le_y})
output.to_csv('submission.csv', index=False)
from google.colab import files
files.download('submission.csv')